# Abstract Algebras

## References

* [Group Explorer](https://nathancarter.github.io/group-explorer/index.html) -- Visualization software for the abstract algebra classroom
* [Groupprops, The Group Properties Wiki (beta)](https://groupprops.subwiki.org/wiki/Main_Page)
* [Klein four-group, V4](https://en.wikipedia.org/wiki/Klein_four-group)
* [Cyclic group](https://en.wikipedia.org/wiki/Cyclic_group)

## Algebra Definition

In [1]:
import itertools as it

class Algebra:
    '''An abstract algbra with a finite number of elements and an addition table.'''
    
    def __init__(self, name, description, element_names, addition_table):
        self.name = name
        self.desc = description
        self.elements = element_names
        self.table = addition_table
        # For efficiency, calculate the headers up front
        self.col_header = self.table[0]
        self.row_header = [row[0] for row in self.table]
        
    def __str__(self):
        return f"<{self.__class__.__name__}: {self.name}, {self.desc}>"
    
    def __repr__(self):
        return f"{self.__class__.__name__}('{self.name}', '{self.desc}', {self.elements}, {self.table})"
    
    def inverse(self, element):
        elem_index = self.elements.index(element)
        row_index = self.row_header.index(elem_index)
        col_index = self.table[row_index].index(0)
        return self.elements[self.col_header[col_index]]
    
    def addition_table_with_names(self):
        return [[self.elements[x] for x in row] for row in self.table]
        
    def add(self, r, c):
        '''Return the sum of elements, r & c.
        The inputs, r & c, can be numbers or strings, but if either
        input is a number, then a number will be returned, otherwise
        the sum's element name (a string) will be returned.'''

        # Table lookup requires numbers
        r_ = r; c_ = c
        str_result = False
        if type(r) == str:
            r_ = self.elements.index(r)
            str_result = True
        if type(c) == str:
            c_ = self.elements.index(c)
            str_result = True
        
        # Lookup the product based on the row & column indices
        row_index = self.row_header.index(r_)
        col_index = self.col_header.index(c_)
        product = self.table[row_index][col_index]
        
        # If either input value was a string, then return a string,
        # otherwise return a number
        if str_result:
            return self.elements[product]
        else:
            return product
        
    def __mul__(self, other):
        '''Return the direct product of this algebra with the input algebra, other.'''
        new_name = self.name + "_x_" + other.name
        new_desc = "Direct product of " + self.name + " & " + other.name
        new_elements = list(it.product(self.elements, other.elements))
        new_table = list()
        for e in new_elements:
            new_row = list()
            for f in new_elements:
                new_row.append(new_elements.index((self.add(e[0], f[0]), other.add(e[1], f[1]))))
            new_table.append(new_row)
        return self.__class__(new_name,
                              new_desc,
                              list([f"{c[0]},{c[1]}" for c in new_elements]),
                              new_table)
    
    def swap(self, a, b):
        '''Change the algbra's definition by swapping the order of two elements, a & b.'''
        elem = self.elements
        i, j = elem.index(a), elem.index(b)
        elem[j], elem[i] = elem[i], elem[j]
        for row in self.table:
            k, m = row.index(i), row.index(j)
            row[k], row[m] = row[m], row[k]
        return None
    
    def elements(self):
        return self.table[0]

# Utilities

<b>make-table</b> helps turn the XML-based tables at Groupprops into a list of lists for use here.

INSTRUCTIONS FOR USE:
1. Copy the table from there and paste it here
1. Find & Replace the strings, "\<row\>" and "\<\/row\>", with nothing
1. Place triple quotes around the result and give it a variable name

In [2]:
def make_table(table_string):
    return [[int(n) for n in row.strip().split(" ")]
            for row in table_string.splitlines()]

def swap_list_items(lst, item1, item2):
    a, b = lst.index(item1), lst.index(item2)
    lst[b], lst[a] = lst[a], lst[b]
    return None

## Group Definition

In [3]:
class Group(Algebra):
    pass

### Klein-4 Group

See this [definition at GitHub](https://github.com/nathancarter/group-explorer/blob/master/groups/V_4.group).

In [4]:
v4 = Group('V4',
           'Klein-4 group',
           ['e',  'h',  'v', 'hv'],
           [[0, 1, 2, 3],
            [1, 0, 3, 2],
            [2, 3, 0, 1],
            [3, 2, 1, 0]]
          )

In [5]:
v4.addition_table_with_names()

[['e', 'h', 'v', 'hv'],
 ['h', 'e', 'hv', 'v'],
 ['v', 'hv', 'e', 'h'],
 ['hv', 'v', 'h', 'e']]

In [6]:
v4.add('h','v')

'hv'

In [7]:
for elem in v4.elements:
    print(f"inv({elem}) = {v4.inverse(elem)}")

inv(e) = e
inv(h) = h
inv(v) = v
inv(hv) = hv


In [8]:
print(v4)

<Group: V4, Klein-4 group>


### Cyclic group of order 4

See this [definition at GitHub](https://github.com/nathancarter/group-explorer/blob/master/groups/Z_4.group).

In [9]:
z4 = Group('Z4',
           'Cyclic group of order 4',
           ['e', 'a', 'a^2', 'a^3'],
           [[0, 1, 2, 3],
            [1, 2, 3, 0],
            [2, 3, 0, 1],
            [3, 0, 1, 2]]
          )

In [10]:
z4.addition_table_with_names()

[['e', 'a', 'a^2', 'a^3'],
 ['a', 'a^2', 'a^3', 'e'],
 ['a^2', 'a^3', 'e', 'a'],
 ['a^3', 'e', 'a', 'a^2']]

In [11]:
print(z4)

<Group: Z4, Cyclic group of order 4>


In [12]:
z4.add(2,2)

0

In [13]:
z4.add('a^2', 'a^3')

'a'

In [14]:
for elem in z4.elements:
    print(f"inv({elem}) = {z4.inverse(elem)}")

inv(e) = e
inv(a) = a^3
inv(a^2) = a^2
inv(a^3) = a


### Symmetric group on 3 letters

See this [definition at GitHub](https://github.com/nathancarter/group-explorer/blob/master/groups/S_3.group). "Another name for this group is "Dihedral group on 3 vertices."

In [15]:
s3 = Group('S3',
           'Symmetric group on 3 letters',
           ['e', 'r', 'r^2', 'f', 'fr', 'rf'],
           [[0, 1, 2, 3, 4, 5],
            [1, 2, 0, 5, 3, 4],
            [2, 0, 1, 4, 5, 3],
            [3, 4, 5, 0, 1, 2],
            [4, 5, 3, 2, 0, 1],
            [5, 3, 4, 1, 2, 0]]
          )

In [16]:
s3.addition_table_with_names()

[['e', 'r', 'r^2', 'f', 'fr', 'rf'],
 ['r', 'r^2', 'e', 'rf', 'f', 'fr'],
 ['r^2', 'e', 'r', 'fr', 'rf', 'f'],
 ['f', 'fr', 'rf', 'e', 'r', 'r^2'],
 ['fr', 'rf', 'f', 'r^2', 'e', 'r'],
 ['rf', 'f', 'fr', 'r', 'r^2', 'e']]

In [17]:
print(s3)

<Group: S3, Symmetric group on 3 letters>


In [18]:
s3.add('fr', 'r^2')

'f'

In [19]:
for elem in s3.elements:
    print(f"inv({elem}) = {s3.inverse(elem)}")

inv(e) = e
inv(r) = r^2
inv(r^2) = r
inv(f) = f
inv(fr) = fr
inv(rf) = rf


In [20]:
s3

Group('S3', 'Symmetric group on 3 letters', ['e', 'r', 'r^2', 'f', 'fr', 'rf'], [[0, 1, 2, 3, 4, 5], [1, 2, 0, 5, 3, 4], [2, 0, 1, 4, 5, 3], [3, 4, 5, 0, 1, 2], [4, 5, 3, 2, 0, 1], [5, 3, 4, 1, 2, 0]])

### Symmetric Group, S3 (not same as S3, above)

This is the [Symmetric group, S3, as specified at Groupprops](https://groupprops.subwiki.org/wiki/Symmetric_group:S3).

W.r.t., how to interpret the addition table, the following quote helps:

[[from Wikipedia](https://groupprops.subwiki.org/wiki/Symmetric_group:S3)] "We portray elements as permutations on the set \{ 1,2,3 \} using the cycle decomposition. The row element is added on the left and the column element on the right, with the assumption of functions written on the left. This means that the column element is applied first and the row element is applied next."

In [21]:
s3x = Group('S3X',
           'Another version of the symmetric group on 3 letters',
           ['()', '(1,2)', '(2,3)', '(1,3)', '(1,2,3)', '(1,3,2)'],
           [[0, 1, 2, 3, 4, 5],
            [1, 0, 4, 5, 2, 3],
            [2, 5, 0, 4, 3, 1],
            [3, 4, 5, 0, 1, 2],
            [4, 3, 1, 2, 5, 0],
            [5, 2, 3, 1, 0, 4]]
          )

In [22]:
s3x.addition_table_with_names()

[['()', '(1,2)', '(2,3)', '(1,3)', '(1,2,3)', '(1,3,2)'],
 ['(1,2)', '()', '(1,2,3)', '(1,3,2)', '(2,3)', '(1,3)'],
 ['(2,3)', '(1,3,2)', '()', '(1,2,3)', '(1,3)', '(1,2)'],
 ['(1,3)', '(1,2,3)', '(1,3,2)', '()', '(1,2)', '(2,3)'],
 ['(1,2,3)', '(1,3)', '(1,2)', '(2,3)', '(1,3,2)', '()'],
 ['(1,3,2)', '(2,3)', '(1,3)', '(1,2)', '()', '(1,2,3)']]

In [23]:
s3x.add('(1,2)', '(2,3)')

'(1,2,3)'

In [24]:
for elem in s3x.elements:
    print(f"inv({elem}) = {s3x.inverse(elem)}")

inv(()) = ()
inv((1,2)) = (1,2)
inv((2,3)) = (2,3)
inv((1,3)) = (1,3)
inv((1,2,3)) = (1,3,2)
inv((1,3,2)) = (1,2,3)


## Z_2 x Z_2 x Z_2

In [25]:
Z2xZ2xZ2 = Group('Z_2 x Z_2 x Z_2',
                 'no description',
                 ['eee', 'aee', 'eae', 'aae', 'eea', 'aea', 'eaa', 'aaa'],
                 [[0, 1, 2, 3, 4, 5, 6, 7],
                  [1, 0, 3, 2, 5, 4, 7, 6],
                  [2, 3, 0, 1, 6, 7, 4, 5],
                  [3, 2, 1, 0, 7, 6, 5, 4],
                  [4, 5, 6, 7, 0, 1, 2, 3],
                  [5, 4, 7, 6, 1, 0, 3, 2],
                  [6, 7, 4, 5, 2, 3, 0, 1],
                  [7, 6, 5, 4, 3, 2, 1, 0]]
                )

In [26]:
Z2xZ2xZ2.addition_table_with_names()

[['eee', 'aee', 'eae', 'aae', 'eea', 'aea', 'eaa', 'aaa'],
 ['aee', 'eee', 'aae', 'eae', 'aea', 'eea', 'aaa', 'eaa'],
 ['eae', 'aae', 'eee', 'aee', 'eaa', 'aaa', 'eea', 'aea'],
 ['aae', 'eae', 'aee', 'eee', 'aaa', 'eaa', 'aea', 'eea'],
 ['eea', 'aea', 'eaa', 'aaa', 'eee', 'aee', 'eae', 'aae'],
 ['aea', 'eea', 'aaa', 'eaa', 'aee', 'eee', 'aae', 'eae'],
 ['eaa', 'aaa', 'eea', 'aea', 'eae', 'aae', 'eee', 'aee'],
 ['aaa', 'eaa', 'aea', 'eea', 'aae', 'eae', 'aee', 'eee']]

In [27]:
for elem in Z2xZ2xZ2.elements:
    print(f"inv({elem}) = {Z2xZ2xZ2.inverse(elem)}")

inv(eee) = eee
inv(aee) = aee
inv(eae) = eae
inv(aae) = aae
inv(eea) = eea
inv(aea) = aea
inv(eaa) = eaa
inv(aaa) = aaa


### Example

In [28]:
foo = '''         0 1 2 3 4 5 6 7 
         1 0 3 2 5 4 7 6 
         2 3 0 1 6 7 4 5 
         3 2 1 0 7 6 5 4 
         4 5 6 7 0 1 2 3 
         5 4 7 6 1 0 3 2 
         6 7 4 5 2 3 0 1 
         7 6 5 4 3 2 1 0 '''

In [29]:
z2_x_z2_x_z2 = make_table(foo)
z2_x_z2_x_z2

[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 3, 2, 5, 4, 7, 6],
 [2, 3, 0, 1, 6, 7, 4, 5],
 [3, 2, 1, 0, 7, 6, 5, 4],
 [4, 5, 6, 7, 0, 1, 2, 3],
 [5, 4, 7, 6, 1, 0, 3, 2],
 [6, 7, 4, 5, 2, 3, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]

In [30]:
fu = "<e,e,e> <a,e,e> <e,a,e> <a,a,e> <e,e,a> <a,e,a> <e,a,a> <a,a,a>"

In [31]:
fubar = fu.split(" ")
fubar

['<e,e,e>',
 '<a,e,e>',
 '<e,a,e>',
 '<a,a,e>',
 '<e,e,a>',
 '<a,e,a>',
 '<e,a,a>',
 '<a,a,a>']

## Direct Products

In [50]:
z2 = Group('Z2',
           'Cyclic group of order 2',
           ['E', 'A'],
           [[0, 1],
            [1, 0]]
          )

z3 = Group('Z3',
           'Cyclic group of order 3',
           ['E', 'A', 'B'],
           [[0, 1, 2],
            [1, 2, 0],
            [2, 0, 1]]
          )

In [51]:
def direct_product(g1, g2):
    new_elements = list(it.product(g1.elements, g2.elements))
    new_table = list()
    for e1 in new_elements:
        new_row = list()
        for e2 in new_elements:
            new_row.append(new_elements.index((g1.add(e1[0], e2[0]), g2.add(e1[1], e2[1]))))
        new_table.append(new_row)
    return list([f"{c[0]},{c[1]}" for c in new_elements])

In [52]:
z3_x_z3 = z3 * z3
z3_x_z3

Group('Z3_x_Z3', 'Direct product of Z3 & Z3', ['E,E', 'E,A', 'E,B', 'A,E', 'A,A', 'A,B', 'B,E', 'B,A', 'B,B'], [[0, 1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 0, 4, 5, 3, 7, 8, 6], [2, 0, 1, 5, 3, 4, 8, 6, 7], [3, 4, 5, 6, 7, 8, 0, 1, 2], [4, 5, 3, 7, 8, 6, 1, 2, 0], [5, 3, 4, 8, 6, 7, 2, 0, 1], [6, 7, 8, 0, 1, 2, 3, 4, 5], [7, 8, 6, 1, 2, 0, 4, 5, 3], [8, 6, 7, 2, 0, 1, 5, 3, 4]])

In [53]:
z3_x_z3.elements

['E,E', 'E,A', 'E,B', 'A,E', 'A,A', 'A,B', 'B,E', 'B,A', 'B,B']

In [54]:
z3_x_z3.table

[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 0, 4, 5, 3, 7, 8, 6],
 [2, 0, 1, 5, 3, 4, 8, 6, 7],
 [3, 4, 5, 6, 7, 8, 0, 1, 2],
 [4, 5, 3, 7, 8, 6, 1, 2, 0],
 [5, 3, 4, 8, 6, 7, 2, 0, 1],
 [6, 7, 8, 0, 1, 2, 3, 4, 5],
 [7, 8, 6, 1, 2, 0, 4, 5, 3],
 [8, 6, 7, 2, 0, 1, 5, 3, 4]]

In [58]:
for elem in z3_x_z3.elements:
    print(f"inv({elem}) = {z3_x_z3.inverse(elem)}")

inv(E,E) = E,E
inv(E,A) = E,B
inv(E,B) = E,A
inv(A,E) = B,E
inv(A,A) = B,B
inv(A,B) = B,A
inv(B,E) = A,E
inv(B,A) = A,B
inv(B,B) = A,A


In [55]:
z2_x_z2_x_z2 = z2 * z2 * z2

In [56]:
z2_x_z2_x_z2.elements

['E,E,E', 'E,E,A', 'E,A,E', 'E,A,A', 'A,E,E', 'A,E,A', 'A,A,E', 'A,A,A']

In [57]:
z2_x_z2_x_z2.table

[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 3, 2, 5, 4, 7, 6],
 [2, 3, 0, 1, 6, 7, 4, 5],
 [3, 2, 1, 0, 7, 6, 5, 4],
 [4, 5, 6, 7, 0, 1, 2, 3],
 [5, 4, 7, 6, 1, 0, 3, 2],
 [6, 7, 4, 5, 2, 3, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]

In [59]:
for elem in z2_x_z2_x_z2.elements:
    print(f"inv({elem}) = {z2_x_z2_x_z2.inverse(elem)}")

inv(E,E,E) = E,E,E
inv(E,E,A) = E,E,A
inv(E,A,E) = E,A,E
inv(E,A,A) = E,A,A
inv(A,E,E) = A,E,E
inv(A,E,A) = A,E,A
inv(A,A,E) = A,A,E
inv(A,A,A) = A,A,A
